# 02. EEG Preprocessing


In [12]:
# 01. CONFIGURAÇÃO INICIAL (PARÂMETROS E DIRETÓRIOS)

import os
import mne
import warnings
import numpy as np
from mne_icalabel import label_components
from autoreject import get_rejection_threshold

# Configuração de Ambiente
mne.set_log_level('WARNING')
warnings.filterwarnings('ignore')

# Definição de Caminhos
DIR_RAW = '../data/raw/' 
DIR_ICA_OBJ = '../data/interim/ica_objects/'
DIR_CLEANED = '../data/interim/cleaned_data/'

# Criar diretórios se não existirem
for d in [DIR_ICA_OBJ, DIR_CLEANED]:
    os.makedirs(d, exist_ok=True)

# Parâmetros Gerais
RANDOM_STATE = 97
L_FREQ = 1.0       # Passa-alta
H_FREQ = 40.0      # Passa-baixa
NOTCH_FREQ = 60.0  # Notch

# Lista oficial de 18 canais (Excluindo linha média Fz, Cz, Pz)
CANAIS_MANTER = [
    'Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8',
    'C3', 'C4', 'T3', 'T4', 'T5', 'P3', 'P4', 'T6',
    'O1', 'O2', 'A1', 'A2'
]

# Pacientes com erro crítico na coleta que devem ser ignorados
ARQUIVOS_EXCLUIDOS = ['control_14-sem marcacoes', 'TEA_M_22_1']

# Dicionário de Limpeza (conferência manual)
# O algoritmo consultará esta lista para sobrescrever a decisão da IA
AJUSTES_MANUAIS = {
    # CONTROLES
    'control_12_basal_pos': {'manter': [10, 14], 'remover': []},
    'control_12_basal_pre': {'manter': [14], 'remover': [4, 5]},
    'control_12_faces':     {'manter': [14], 'remover': []},
    'control_13_basal_pre': {'manter': [10], 'remover': [4]},
    'control_13_faces':     {'manter': [12], 'remover': []},
    'control_35_basal_pre': {'manter': [8, 15], 'remover': []},
    'control_37_basal_pos': {'manter': [14], 'remover': []},
    'control_39_faces':     {'manter': [8], 'remover': []},
    'control_43_basal_pos': {'manter': [], 'remover': [12]},
    'control_48_faces':     {'manter': [11], 'remover': []},
    'control_56_basal_pre': {'manter': [], 'remover': [1, 2]},
    'control_57_basal_pre': {'manter': [10], 'remover': []},
    'control_58_basal_pos': {'manter': [3], 'remover': []},
    'control_58_basal_pre': {'manter': [], 'remover': [4, 15]},
    'control_58_faces':     {'manter': [15], 'remover': []},
    'control_61_basal_pos': {'manter': [5], 'remover': []},
    
    # TEA
    'TEA_G_26_basal_pos':   {'manter': [9, 13], 'remover': []},
    'TEA_G_28_faces':       {'manter': [7, 11], 'remover': []},
    'TEA_G_30_faces':       {'manter': [5], 'remover': []}, 
    'TEA_L_16_faces':       {'manter': [6, 7], 'remover': []},
    'TEA_L_25_basal_pre':   {'manter': [4, 9], 'remover': []},
    'TEA_L_27_basal_pos':   {'manter': [], 'remover': [10]},
    'TEA_L_27_basal_pre':   {'manter': [3], 'remover': []},
    'TEA_L_27_faces':       {'manter': [], 'remover': [9]},
    'TEA_L_29_basal_pre':   {'manter': [], 'remover': [12]},
    'TEA_L_29_faces':       {'manter': [14], 'remover': []},
    'TEA_M_15_basal_pre':   {'manter': [6], 'remover': []},
}

print("Ambiente, Pastas e Parâmetros Configurados com Sucesso.")

Ambiente, Pastas e Parâmetros Configurados com Sucesso.


In [13]:
# 02. PRÉ-PROCESSAMENTO COMPLETO

import time

arquivos_edf = sorted([f for f in os.listdir(DIR_RAW) if f.endswith('.edf')])
total_arquivos = len(arquivos_edf)

print(f"Iniciando Pipeline Completo para {total_arquivos} arquivos brutos...\n")

stats = {'sucesso': 0, 'excluidos': 0, 'erros': 0}
inicio_pipeline = time.time()

for idx, arquivo in enumerate(arquivos_edf, 1):
    paciente_id = arquivo.replace('.edf', '')
    
    if paciente_id in ARQUIVOS_EXCLUIDOS:
        print(f"[{idx:02d}/{total_arquivos}] -> {paciente_id}: Excluído por falha de coleta.")
        stats['excluidos'] += 1
        continue
        
    print(f"\n[{idx:02d}/{total_arquivos}] Abrindo paciente: {paciente_id}")
    caminho_entrada = os.path.join(DIR_RAW, arquivo)
    
    try:
        # 1. Carregamento dos dados bruto e filtragem básica
        raw = mne.io.read_raw_edf(caminho_entrada, preload=True, encoding='latin1', verbose=False)
        
        rename_map = {ch: ch.replace('EEG ', '').replace('-Ref', '').strip() for ch in raw.ch_names}
        raw.rename_channels(rename_map)
        raw.pick([ch for ch in CANAIS_MANTER if ch in raw.ch_names], verbose=False)
        
        raw.set_montage(mne.channels.make_standard_montage('standard_1020'), on_missing='ignore') # Sistema de Montagem 10-20
        raw.set_eeg_reference('average', projection=False, verbose=False)
        raw.filter(l_freq=L_FREQ, h_freq=H_FREQ, fir_design='firwin', verbose=False) # Passa-Banda
        raw.notch_filter(freqs=NOTCH_FREQ, verbose=False) # Filtro Notch
        
        # 2. Segmentação (crop)
        tempos = {'B1_INICIO': None, 'B1_FIM': None, 'B2_INICIO': None, 'B2_FIM': None}
        primeiro_estimulo = None 
        
        for ann in raw.annotations:
            desc, t = ann['description'].upper(), ann['onset']
            if ('BASAL 1 INICIO' in desc or 'BASAL 1 INCIO' in desc) and tempos['B1_INICIO'] is None: tempos['B1_INICIO'] = t
            elif 'BASAL 1 FINAL' in desc or 'BASAL 1 PARTE 2:2' in desc: tempos['B1_FIM'] = t
            elif ('BASAL 2 INICIO' in desc or 'BASAL 2 INCIO' in desc) and tempos['B2_INICIO'] is None: tempos['B2_INICIO'] = t
            elif 'BASAL 2 FINAL' in desc: tempos['B2_FIM'] = t
            if desc in ['FF', 'FN', 'FR', 'VIDEO', 'PROJECAO'] and primeiro_estimulo is None: primeiro_estimulo = t

        # Resgate de marcadores faltantes (ex: control_37)
        if tempos['B1_INICIO'] is None and tempos['B1_FIM'] is None:
            if primeiro_estimulo is not None:
                tempos['B1_FIM'] = max(0.0, primeiro_estimulo - 10.0)
                tempos['B1_INICIO'] = max(0.0, tempos['B1_FIM'] - 180.0)
            else:
                raise ValueError("Sem marcadores de Basal ou Tarefa.")

        # Imputação de bordas faltantes assumindo 3 min (180s)
        if tempos['B1_INICIO'] is not None and tempos['B1_FIM'] is None: tempos['B1_FIM'] = tempos['B1_INICIO'] + 180.0
        if tempos['B1_FIM'] is not None and tempos['B1_INICIO'] is None: tempos['B1_INICIO'] = max(0.0, tempos['B1_FIM'] - 180.0)
        if tempos['B2_INICIO'] is not None and tempos['B2_FIM'] is None: tempos['B2_FIM'] = min(raw.times[-1], tempos['B2_INICIO'] + 180.0)
        if tempos['B2_FIM'] is not None and tempos['B2_INICIO'] is None: tempos['B2_INICIO'] = max(tempos['B1_FIM'], tempos['B2_FIM'] - 180.0)
            
        # Criação dos blocos físicos na memória
        blocos = {
            'basal_pre': raw.copy().crop(tmin=tempos['B1_INICIO'], tmax=tempos['B1_FIM']),
            'faces': raw.copy().crop(tmin=tempos['B1_FIM'], tmax=tempos['B2_INICIO']),
            'basal_pos': raw.copy().crop(tmin=tempos['B2_INICIO'], tmax=tempos['B2_FIM'])
        }

        # 3. Limpeza avançada para cada bloco
        for nome_bloco, raw_bloco in blocos.items():
            id_unico = f"{paciente_id}_{nome_bloco}"
            caminho_final_bloco = os.path.join(DIR_CLEANED, f"{id_unico}-cleaned-raw.fif")
                          
            print(f" Limpando bloco: {nome_bloco}...")
            
            # Autoreject Dinâmico
            epochs_dummy = mne.make_fixed_length_epochs(raw_bloco, duration=1.0, preload=True, verbose=False)
            n_epochs = len(epochs_dummy)
            if n_epochs < 2:
                limiares_rejeicao = None
            else:
                cv_dinamico = max(2, min(n_epochs, 5))
                reject_thresholds = get_rejection_threshold(epochs_dummy, decim=1, random_state=RANDOM_STATE, cv=cv_dinamico, verbose=False)
                limiares_rejeicao = {'eeg': reject_thresholds['eeg']}
            del epochs_dummy
            
            # Treinamento ICA
            ica = mne.preprocessing.ICA(n_components=15, random_state=RANDOM_STATE, max_iter='auto', method='fastica')
            ica.fit(raw_bloco, reject=limiares_rejeicao, tstep=1.0, verbose=False)
            
            # ICLabel - classificação automática (considerando regra dos 80%)
            ic_labels = label_components(raw_bloco, ica, method='iclabel')
            exclusao_ia = set(i for i, (label, prob) in enumerate(zip(ic_labels['labels'], ic_labels['y_pred_proba'])) if not (label == 'brain' and prob >= 0.80))
                    
            # Auditoria ("correção" com o dicionário manual)
            if id_unico in AJUSTES_MANUAIS:
                regras = AJUSTES_MANUAIS[id_unico]
                for m in regras['manter']: exclusao_ia.discard(m)
                for r in regras['remover']: exclusao_ia.add(r)
                    
            # Aplicação final e salvamento
            ica.exclude = list(exclusao_ia)
            raw_limpo = ica.apply(raw_bloco.copy(), verbose=False)
            
            ica.save(os.path.join(DIR_ICA_OBJ, f"{id_unico}-ica.fif"), overwrite=True, verbose=False)
            raw_limpo.save(caminho_final_bloco, overwrite=True, verbose=False)
            
        stats['sucesso'] += 1
        
    except Exception as e:
        print(f" Erro em {paciente_id}: {str(e)}")
        stats['erros'] += 1

tempo_total = (time.time() - inicio_pipeline) / 60
print("\n" + "="*50)
print("PIPELINE DE PRÉ-PROCESSAMENTO FINALIZADO!")
print(f"Tempo total: {tempo_total:.2f} minutos.")
print(f"Pacientes processados com sucesso: {stats['sucesso']}")
print(f"Excluídos: {stats['excluidos']}")
print(f"Erros: {stats['erros']}")
print(f"Dados limpos gerados e disponíveis em: {DIR_CLEANED}")
print("="*50)

Iniciando Pipeline Completo para 44 arquivos brutos...


[01/44] Abrindo paciente: TEA_G_26
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[02/44] Abrindo paciente: TEA_G_28
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[03/44] Abrindo paciente: TEA_G_30
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[04/44] Abrindo paciente: TEA_G_40
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[05/44] Abrindo paciente: TEA_L_16
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[06/44] Abrindo paciente: TEA_L_17
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[07/44] Abrindo paciente: TEA_L_24
 Limpando bloco: basal_pre...
 Limpando bloco: faces...
 Limpando bloco: basal_pos...

[08/44] Abrindo paciente: TEA_L_25
 Limpando bloco: basal_pre...
 Limpando bloco: faces..